In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mean, stdev
import matplotlib
from rlhfutils.data import load_wgpt, augment_data, load_apfarm, load_stack, load_rlcd
from dataclasses import dataclass
from transformers import AutoTokenizer
from collections import Counter
import random
from statistics import mean, stdev
from random import choices
import seaborn as sns

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
toker = AutoTokenizer.from_pretrained("../models/rewards/apffgoodcut2/")

In [ ]:
def sigint(acc, tot, runs):
    sampdist = [1]*int(acc*tot) + [0]*int((1-acc)*tot)
    mtots = []
    for i in range(10000):
        cs =  choices(sampdist, k=len(sampdist))
        mtots.append(mean(cs))
    return mean(mtots), stdev(mtots)

def siginttwo(acc1, acc2, tot, runs):
    sampdist = [1]*int(acc1*tot) + [0]*int((1-acc1)*tot)
    sampdist2 = [1]*int(acc2*tot) + [0]*int((1-acc2)*tot)
    mtots = []
    for i in range(10000):
        cs =  choices(sampdist, k=len(sampdist))
        cs2 =  choices(sampdist2, k=len(sampdist2))
        mtots.append(int(mean(cs) > mean(cs2)))
    return mean(mtots), stdev(mtots)

In [ ]:
siginttwo(0.34, 0.30, 500, 100000)

In [ ]:
# Custom aggregation function to collect values into a list and sort by index
def collect_and_sort(series):
    return list(series.sort_index())

def make_statdf(cdf):
    # Group by 'uid' and aggregate
    new_df = cdf.groupby('uid').agg({
        'rew_j': collect_and_sort,
        'rew_k': collect_and_sort
    }).reset_index()
    new_df['diffs'] = [[r['rew_j'][i] - r['rew_k'][i] for i in range(len(r['rew_j']))] for _, r in new_df.iterrows()]
    return new_df

def lenscat(idf):
    return plt.hist2d(idf['vars'], idf['confs'], bins=(50, 50), norm=matplotlib.colors.LogNorm())

def lenheur(idf):
    return  (idf['tj']>idf['tk']).mean()

def alltoks(col):
    atoks = []
    for c in col:
        atoks.extend(list(set(toker(c).input_ids)))
    return [toker.decode(d) for d in atoks]

def companalysis(indf, thresh):
    good = Counter(alltoks(indf['rj']))
    bad = Counter(alltoks(indf['rk']))

    total = good+bad
    total = {token: count for token, count in total.items() if count >= thresh}
    
    # Compute the conditional probability P(Class|Token) for each token and each class
    good_token_prob = {token: good.get(token, 0) / total[token] for token in total}
    bad_token_prob = {token: bad.get(token, 0) / total[token] for token in total}
    
    # Find top tokens for each class
    top_good_tokens = sorted(good_token_prob.items(), key=lambda x: x[1], reverse=True)[:5]
    top_bad_tokens = sorted(bad_token_prob.items(), key=lambda x: x[1], reverse=True)[:5]
    
    print("Top tokens with probabilities for 'good' class:")
    for token, prob in top_good_tokens:
        print(f"{token}: {prob:.4f}")
    
    print("Top tokens with probabilities for 'bad' class:")
    for token, prob in top_bad_tokens:
        print(f"{token}: {prob:.4f}")

def lenscat(idf):
    plt.hist2d(idf['vars'], idf['confs'], bins=(50, 50), norm=matplotlib.colors.LogNorm())

def lenheur(idf):
    return  (idf['tj']>idf['tk']).mean()

def alltoks(col):
    atoks = []
    for c in col:
        atoks.extend(list(set(toker(c).input_ids)))
    return [toker.decode(d) for d in atoks]

def companalysis(indf, thresh):
    good = Counter(alltoks(indf['rj']))
    bad = Counter(alltoks(indf['rk']))

    total = good+bad
    total = {token: count for token, count in total.items() if count >= thresh}
    
    # Compute the conditional probability P(Class|Token) for each token and each class
    good_token_prob = {token: good.get(token, 0) / total[token] for token in total}
    bad_token_prob = {token: bad.get(token, 0) / total[token] for token in total}
    
    # Find top tokens for each class
    top_good_tokens = sorted(good_token_prob.items(), key=lambda x: x[1], reverse=True)[:5]
    top_bad_tokens = sorted(bad_token_prob.items(), key=lambda x: x[1], reverse=True)[:5]
    
    print("Top tokens with probabilities for 'good' class:")
    for token, prob in top_good_tokens:
        print(f"{token}: {prob:.4f}")
    
    print("Top tokens with probabilities for 'bad' class:")
    for token, prob in top_bad_tokens:
        print(f"{token}: {prob:.4f}")
        
@dataclass
class tmpdata: 
    mix_ratio: str
    rand_ratio:  str

def procn(ntmp, tdset):
    try:
        ntmp['isrand'] = [tdset[r['uid']]['isrand'] for _, r in ntmp.iterrows()]
    except: 
        print("no randd")
    ntmp['rj'] = [tdset[r['uid']]['response_j'] for _, r in ntmp.iterrows()]
    ntmp['rk'] = [tdset[r['uid']]['response_k'] for _, r in ntmp.iterrows()]
    ntmp['tj'] = [len(toker(r['rj']).input_ids) for _, r in ntmp.iterrows()]
    ntmp['tk'] = [len(toker(r['rk']).input_ids) for _, r in ntmp.iterrows()]
    return ntmp
    
def getprocddata(statdf, dset, da=False):
    print("load ", dset)
    # if we're doing WGPT
    if dset=="wgpt":
        train_dataset = load_wgpt()[0]
    elif dset=="apf":
        train_dataset = load_apfarm("gpt4")[0]
    elif dset=="rlcd":
        train_dataset = load_rlcd()[0]
    elif dset=="stack":
        train_dataset = load_stack()[0]
    if da:
        train_dataset = augment_data(train_dataset, tmpdata(**{"mix_ratio":0, "rand_ratio":0.2}))
    train_dataset = augment_data(train_dataset, tmpdata(**{"mix_ratio":0, "rand_ratio":0}))
    print("start processing, will take a minute")
    return procn(statdf, train_dataset)

cartopaths = {
    "wgpt":"../trl-general/carto_outs/webgpt_carto.jsonl",
    "stack":"../trl-general/carto_outs/stack_carto.jsonl",
    "rlcd":"../trl-general/carto_outs/rlcd_carto.jsonl",
    "apf":"../trl-general/carto_outs/apf_carto.jsonl"
}

def load_carto(dsname, da=False, getprocs=True):
    if getprocs:
        cartdf = pd.read_json(cartopaths[dsname], orient='records', lines=True)
    else:
        cartdf = pd.read_json(dsname, orient='records', lines=True)
    ndf = make_statdf(cartdf)
    ndf['confs'] = [mean(m[1:]) for m in ndf['diffs']]
    ndf['vars'] = [stdev(m[1:]) for m in ndf['diffs']]
    if getprocs:
        return getprocddata(ndf, dsname, da)
    else:
        return ndf

def heurscatter(dset, interv, col='confs'):
    bottom = int(min(dset[col]))
    top = int(max(dset.confs))
    xvals = []
    yvals = []
    dist = []
    for i in range(1, int((top-bottom)/interv)):
        curx = bottom + (interv)*i
        tmpslice = dset.loc[(dset[col]<curx) & (dset[col]>curx-interv)]
        xvals.append(curx)
        yvals.append(lenheur(tmpslice))
        dist.append(len(tmpslice))
    return xvals, yvals, dist

In [ ]:
rlcd = load_carto("rlcd")

In [ ]:
wgpt = load_carto("wgpt")
stack = load_carto("stack")
#rlcd = load_carto("rlcd")
#apfarm = load_carto("apf")

In [ ]:
import pickle 

In [ ]:
with open('cartodata.pkl', 'wb') as f:
    pickle.dump((wgpt, stack, rlcd), f)

In [ ]:
with open('cartodata.pkl', 'rb') as f:
    (w, s, r) = pickle.load(f)

In [ ]:
def plotscatter(dset, interv, col, title):
    x, y, d = heurscatter(dset, interv, col)
    fig, ax = plt.subplots()
    #ax.title.set_title(title) 
    ax.set_xlabel('Confidence')
    ax.set_ylabel('Length Heuristic')
    ax.set_title(title)
    ax.scatter(x, y)
    #plt.savefig(title+col+"DClenheur.pdf", format="pdf", bbox_inches="tight")
    return fig

f = plotscatter(stack, 0.1, 'confs', "stack")

In [ ]:
def plotscatter(dset, interv, col, title, ax):
    x, y, d = heurscatter(dset, interv, col)
    
    # Scale the d values to use for point size. You can adjust the scaling factor.
    sizes = [val * 10 for val in d]  # adjust the factor (here, 50) as per your data and preferences
    
    sns.scatterplot(x=x, y=y, size=sizes, sizes=(50, 1000), ax=ax, legend=False)
    ax.set_xlabel('Confidence')
    ax.set_ylabel('Length Heuristic')
    ax.set_title(title)

# Setting the style for a prestigious machine learning conference
sns.set_style("whitegrid")
sns.set_context("talk")  # or "paper" or "talk", depends on how big you want things

# Create a figure and axes for the three scatter plots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Plot each scatter plot on the individual axes
plotscatter(w, 0.2, 'confs', "W-GPT", axs[0])
plotscatter(s, 0.5, 'confs', "Stack", axs[1])
plotscatter(r, 1, 'confs', "RLCD", axs[2])

plt.tight_layout()
# Uncomment the next line if you want to save the figure
plt.savefig("carto_scatters.pdf", format="pdf", bbox_inches="tight")

plt.show()

In [ ]:
f = plotscatter(stack, 0.1, 'confs', "stack")

In [ ]:
# get versions with both cuts and only 1
dsets = {
    "webgpt":wgpt,
    "apf":apfarm,
    "rlcd":rlcd,
    "stack":stack
}
cuts = {
    "webgpt":[-0.2, 0.4],
    "apf":[-0.3, 0.3],
    "rlcd":[-1, 2],
    "stack": [-1,  1]
}
for c in cuts.keys():
    tmp = dsets[c]
    #tmp = tmp[tmp.confs>cuts[c][0]]
    #tmp.uid.to_json("../trl-general/truncvals/"+c+"bad.json", lines=True,  orient='records')
    tmp = tmp[tmp.confs<cuts[c][1]]
    tmp.uid.to_json("../trl-general/truncvals/"+c+"good.json", lines=True,  orient='records')

In [ ]:
lenscat(apfarm)

In [ ]:
tmp = apfarm
tmp = tmp[(tmp.confs>0.25)|(tmp.confs<-0.25)]
print(len(tmp), "/", len(apfarm))
lenscat(tmp)

In [ ]:
l1 = list(pd.read_json("../trl-general/truncvals/rlcdboth.json", orient='records', lines=True)[0])

In [ ]:
l2 = list(tmp['uid'])

In [ ]:
len(stack)

In [ ]:
lenheur(rlcd[rlcd.confs<0.5])

In [ ]:
lenheur(stack.loc[(stack.confs<0)])

In [ ]:
lenscat(stack)

In [ ]:
len(tmp)

In [ ]:
lonly = load_carto("../trl-general/carto_outs/.jsonl", False, False)

In [ ]:
lenscat(lonly)

In [ ]:
lonly = load_carto("../trl-general/carto_outs/wgptbothandda.jsonl", False, False)

In [ ]:
lenscat(lonly)

In [ ]:
rlcdbase = load_carto("../trl-general/carto_outs/rlcd_carto.jsonl", False, False)

In [ ]:
sdf = pd.read_json("../trl-general/carto_outs/leftonlyv3rm.jsonl", orient='records', lines=True)
sdf = make_statdf(sdf)

In [ ]:
#lsanity = load_carto("../trl-general/carto_outs/midcutsanity.jsonl", False, False)
rsanity = load_carto("../trl-general/carto_outs/rlcdgoodcutv3.jsonl", False, False)

In [ ]:
lenscat(rsanity)

In [ ]:
rsani.vars.median()

In [ ]:
rsanity.vars.median()

In [ ]:
rlcdbase.confs.abs().mean()

In [ ]:
sanuids = pd.read_json("../trl-general/truncvals/rlcdbothcutminisanity.json", orient='records', lines=True)

In [ ]:
lenscat(rlcdbase[rlcdbase.uid.isin(sanuids[0])])

In [ ]:
lenscat(rsanity)

In [ ]:
midt.vars.mean()

In [ ]:
tmp.uid.to_json("../trl-general/truncvals/"+"leftonly.json", lines=True,  orient='records')

In [ ]:
tmp = stack
tmp = tmp[tmp.confs<1]
tmp = tmp[tmp.confs>-1]
print(len(tmp))
lenscat(tmp)

In [ ]:
truncuids = tmp.uid
truncuids.to_json("../trl-general/truncvals/rlcdbadhoriz.json", lines=True,  orient='records')

In [ ]:
# files where I'm storing truncvals
fnames = ['wgboth', 'wgbadonly', 'rlcdbadonly', 'rlcdboth', 'apfboth', 'apfbadonly']

In [ ]:
max(list(pd.read_json("../trl-general/truncvals/webgptboth.json", lines=True, orient='records')[0]))

In [ ]:
#companalysis(hard[hard.isrand==0], 25)

In [ ]:
lenscat(ndf[ndf.confs>ndf.vars*2.5])

In [ ]:
lenscat(hard)

In [ ]:
# when skipping first 
easy = ndf[ndf['confs']-4>(ndf['vars']*1.6)]
ambig = ndf[ndf['confs']-4<(ndf['vars']*1.6)]
hard = ambig[ambig['confs']-10<(ambig['vars']*-0.5)]
ambig = ambig[ambig['confs']-10>(ambig['vars']*-0.5)]
# when not skipping first
# easy = ndf[ndf['confs']-2.5>(ndf['vars']*1.6)]
# ambig = ndf[ndf['confs']-2.5<(ndf['vars']*1.6)]
# hard = ambig[ambig['confs']-10<(ambig['vars']*-2)]
# ambig = ambig[ambig['confs']-10>(ambig['vars']*-2)]

In [ ]:
lenheur(hard)

In [ ]:
plt.hist(ndf['vars'])

In [ ]:
hard.isrand.mean()

In [ ]:
lenscat(ndf[(ndf.tj-ndf.tk)>40])

In [ ]:
ndf.loc[0].rew_j

In [ ]:
ind = 4
print(easy.iloc[ind]['rj'])
print("____")
print(easy.iloc[ind]['rk'])

In [ ]:
companalysis(hard, 20)

In [ ]:
lenscat(easy)

In [ ]:
lenheur(ambig)

In [ ]:
lenscat(hard[ndf['tj']>ndf['tk']])

In [ ]:
plt.scatter(ndf['vars'], ndf['confs'])

In [ ]:
ndf = ndf.reset_index()

In [ ]:
plt.hist2d(ndf['vars'], ndf['confs'], bins=(50, 50), norm=matplotlib.colors.LogNorm())